# 正規化層

入力されたデータを正規化して返す層。

入力されたデータが特定の統計量（平均・分散）に従うように正規化される。  
統計量は学習可能なパラメータである。

平均0, 分散1への正規化+シフトとも見られる。

統計量を求める軸によって色々な種類がある。
- バッチ正規化
- 層正規化
- インスタンス正規化

pytorchで挙動を見ながら理解していこう。

https://pytorch.org/docs/stable/nn.html#normalization-layers

In [1]:
import torch
from torch import nn


---

## バッチ正規化

*Batch Normalization*

バッチ方向に正規化を行う。  
学習時と推論時で挙動が変わる。

https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html

### 学習

学習時の挙動を見てみよう。  
まず適当なデータを用意する。

In [2]:
batch_size = 2
d = 5
x = torch.arange(batch_size * d).reshape(batch_size, d).to(torch.float32)
x

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

pytorchではバッチ方向は0番目の次元になる。

バッチ正規化層を作る。

In [3]:
norm = nn.BatchNorm1d(d)
norm.train();

データを突っ込んでみよう。

In [4]:
y = norm(x)
y

tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
        [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000]],
       grad_fn=<NativeBatchNormBackward0>)

平均0, 分散1に正規化された。  
これはpytorchが設定した初期値であって、学習によって変化する。

In [5]:
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


学習時はミニバッチ前提なので、例えばバッチサイズ1のデータを入れるとエラーが出る。

In [6]:
x[:1]

tensor([[0., 1., 2., 3., 4.]])

In [7]:
try:
    y = norm(x[:1])
except Exception as e:
    print(e)

Expected more than 1 value per channel when training, got input size torch.Size([1, 5])


### 推論

推論時は挙動が変わる。

学習時と同じようにしてしまうと、バッチ内のデータによって結果が変わってしまう。サンプリング方法によって推論結果が変わるのはマズイ。  
後は、バッチサイズ1の時も困る。

推論時は、学習データ全体の統計量で正規化する。  
学習時に、特徴量ごとの統計量を記録しておく。

みてみよう。

In [8]:
norm = nn.BatchNorm1d(d)
norm.eval()
y = norm(x)
y

tensor([[0.0000, 1.0000, 2.0000, 3.0000, 4.0000],
        [5.0000, 6.0000, 7.0000, 8.0000, 9.0000]],
       grad_fn=<NativeBatchNormBackward0>)

同じデータが出力された。  
これは、パラメータに加え、学習データ全体の統計量も平均0, 分散1として初期化されているから。

In [9]:
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.])),
             ('running_mean', tensor([0., 0., 0., 0., 0.])),
             ('running_var', tensor([1., 1., 1., 1., 1.])),
             ('num_batches_tracked', tensor(0))])

`running_mean`と`running_var`が学習データ全体の統計量。  
これらは学習モードで演算を行うと更新される。

In [10]:
norm.train()
y = norm(x)
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.])),
             ('running_mean',
              tensor([0.2500, 0.3500, 0.4500, 0.5500, 0.6500])),
             ('running_var', tensor([2.1500, 2.1500, 2.1500, 2.1500, 2.1500])),
             ('num_batches_tracked', tensor(1))])

In [11]:
print('mean:', x.mean(dim=0))
print('var:', x.var(dim=0))

mean: tensor([2.5000, 3.5000, 4.5000, 5.5000, 6.5000])
var: tensor([12.5000, 12.5000, 12.5000, 12.5000, 12.5000])


入力したデータの平均と分散が記録された。  
`momentum=0.1`がデフォルトで設定されているので、完全には一致していない。

これでもう一度推論モードにしてみると、今記録した統計量で演算が行われる。

In [12]:
norm.eval()
y = norm(x)
y

tensor([[-0.1705,  0.4433,  1.0571,  1.6709,  2.2847],
        [ 3.2395,  3.8533,  4.4671,  5.0808,  5.6946]],
       grad_fn=<NativeBatchNormBackward0>)

確かめてみよう。

In [13]:
mean = norm.state_dict()['bias'] # 平均（パラメータ）
var = norm.state_dict()['weight'] # 分散（パラメータ）
data_mean = norm.state_dict()['running_mean'] # 平均（学習データ）
data_var = norm.state_dict()['running_var'] # 分散（学習データ）
eps = norm.eps # 微小値

((x - data_mean) / torch.sqrt(data_var + eps)) * var + mean

tensor([[-0.1705,  0.4433,  1.0571,  1.6709,  2.2847],
        [ 3.2395,  3.8533,  4.4671,  5.0808,  5.6946]])

ちゃんと同じになったね。

あと、バッチサイズ1の時もちゃんと動く。

In [14]:
norm.eval()
norm(x[:1])

tensor([[-0.1705,  0.4433,  1.0571,  1.6709,  2.2847]],
       grad_fn=<NativeBatchNormBackward0>)

特徴量は2次元でもいい。まとめて正規化される。

In [15]:
batch_size = 2
w = 3
h = 4
x = torch.arange(batch_size * w * h).reshape(batch_size, w, h).to(torch.float32)
x

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])

In [16]:
norm = nn.BatchNorm1d(w)
norm(x)

tensor([[[-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373]],

        [[ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288]]],
       grad_fn=<NativeBatchNormBackward0>)

こういうこと。

In [17]:
mean = x.mean(dim=(0, 2), keepdim=True)
mean

tensor([[[ 7.5000],
         [11.5000],
         [15.5000]]])

In [18]:
var = x.var(dim=(0, 2), keepdim=True, unbiased=False)
var

tensor([[[37.2500],
         [37.2500],
         [37.2500]]])

In [19]:
((x - mean) / torch.sqrt(var + norm.eps))

tensor([[[-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373],
         [-1.2288, -1.0650, -0.9012, -0.7373]],

        [[ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288],
         [ 0.7373,  0.9012,  1.0650,  1.2288]]])

バッチ次元（0番目）の次の次元（1番目）が単位となってまとまるので、それ以外の軸を指定して（`dim=(0, 2)`）計算するというイメージ。

ちなみに3次元以上の特徴量（4次元以上の入力）には対応していない。

In [20]:
batch_size = 2
c = 3
w = 4
h = 5
x = torch.randn(batch_size, c, w, h)

norm = nn.BatchNorm1d(c)
try:
    norm(x)
except Exception as e:
    print(e)

expected 2D or 3D input (got 4D input)


### 特徴量が3次元の場合

主に畳み込み層で使われる。特徴マップの正規化。

https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html

In [21]:
batch_size = 2
c = 3
w = 4
h = 5
x = torch.randn(batch_size, c, w, h)
x.shape

torch.Size([2, 3, 4, 5])

In [22]:
norm = nn.BatchNorm2d(c)
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)


In [23]:
y = norm(x)
y.shape

torch.Size([2, 3, 4, 5])

後ろの二つの次元をまとめれば`BatchNorm1d`でも同じことが出来る。

In [24]:
norm = nn.BatchNorm1d(c)
y_ = norm(x.reshape(batch_size, c, -1)).reshape(batch_size, c, w, h)
torch.equal(y, y_)

True

ちなみに4次元以上の特徴量を扱う`BatchNorm3d`もある。

特徴量の数と対応しているclassをまとめるとこうなる。

| 特徴量の次元数 | 入力の次元数 | class |
| --- | --- | --- |
| 1 | 2 | `BatchNorm1d` |
| 2 | 3 | `BatchNorm1d` |
| 3 | 4 | `BatchNorm2d` |
| 4 | 5 | `BatchNorm3d` |

命名と分け方が謎だ。


---

## 層正規化

*Layer Normalization*

層方向に正規化を行う。層方向というのは、バッチ方向とは異なる方向というか、バッチ次元の次の次元というか。

In [25]:
batch_size = 2
d = 5
x = torch.arange(batch_size * d).reshape(batch_size, d).to(torch.float32)
x

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

In [26]:
norm = nn.LayerNorm(d)
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


In [27]:
y = norm(x)
y

tensor([[-1.4142e+00, -7.0710e-01,  0.0000e+00,  7.0710e-01,  1.4142e+00],
        [-1.4142e+00, -7.0710e-01,  1.7881e-07,  7.0711e-01,  1.4142e+00]],
       grad_fn=<NativeLayerNormBackward0>)

In [28]:
mean = x.mean(dim=-1, keepdim=True)
var = x.var(dim=-1, keepdim=True, unbiased=False)
((x - mean) / torch.sqrt(var + norm.eps))

tensor([[-1.4142, -0.7071,  0.0000,  0.7071,  1.4142],
        [-1.4142, -0.7071,  0.0000,  0.7071,  1.4142]])

In [29]:
batch_size = 2
seq_len = 3
embed_dim = 4
x = torch.randn(batch_size, seq_len, embed_dim)
x

tensor([[[ 0.0109,  0.8088,  2.3607,  1.2876],
         [ 0.7794,  1.4459, -0.9452,  0.2869],
         [ 0.3229,  1.5048,  2.6646,  1.5432]],

        [[ 0.4738,  1.2883, -1.7934, -0.7143],
         [ 1.1579,  0.4236, -1.1131,  1.1659],
         [ 1.0153,  1.0677, -0.2684,  0.6635]]])

In [30]:
norm = nn.LayerNorm(embed_dim)
y = norm(x)
y

tensor([[[-1.3003, -0.3623,  1.4621,  0.2005],
         [ 0.4432,  1.2052, -1.5286, -0.1198],
         [-1.4320, -0.0049,  1.3955,  0.0414]],

        [[ 0.5645,  1.2609, -1.3740, -0.4513],
         [ 0.8068,  0.0162, -1.6383,  0.8153],
         [ 0.7388,  0.8366, -1.6575,  0.0821]]],
       grad_fn=<NativeLayerNormBackward0>)